In [1]:
# 自动计算cell的计算时间
%load_ext autotime

time: 491 µs (started: 2021-08-05 13:24:49 +08:00)


In [4]:
%%bash

# 增加更新
git add *.ipynb

git remote -v

git commit -m '更新 ch12 #1 change Aug 05, 2021'

git push origin master

origin	git@github.com:ustchope/dlwpn.git (fetch)
origin	git@github.com:ustchope/dlwpn.git (push)
[master 6bbb16b] 更新 ch12 #1 change Aug 05, 2021
 2 files changed, 1470 insertions(+), 353 deletions(-)
 create mode 100644 ch12.ipynb


To github.com:ustchope/dlwpn.git
   17af357..6bbb16b  master -> master


time: 4.3 s (started: 2021-08-05 13:25:52 +08:00)


In [2]:
#设置使用的gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

time: 12.2 s (started: 2021-08-05 13:24:51 +08:00)


In [3]:
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 32.6 ms (started: 2021-08-05 13:25:48 +08:00)


# 生成式深度学习

**本章包含**

* 文本生成
* DeepDream
* 神经风格迁移
* 变分自编码器
* 生成对抗网络

人工智能模拟人类思维过程的潜力超越了被动任务（如物体识别）和主要反应性任务（如驾驶汽车）。它很好地扩展到创造性活动中。当我第一次声称在不久的将来，我们消费的大部分文化内容都将在 AI 的大力帮助下创建时，我完全不相信，即使是长期从事机器学习的从业者也是如此。那是在 2014 年。快进三年，怀疑已经消退 - 以令人难以置信的速度。 2015 年夏天，我们被 Google 的 DeepDream 算法逗乐了，该算法将图像变成迷幻的狗眼和幻觉伪像； 2016 年，我们使用 Prisma 应用程序将照片变成各种风格的画作。 2016年夏天，一部实验性短片《太阳之泉》使用长短期记忆编写的剧本进行导演，并配上对白。也许您最近听过由神经网络临时生成的音乐。

诚然，到目前为止，我们从 AI 看到的艺术作品质量相当低。 人工智能远不能与人类编剧、画家和作曲家匹敌。 但取代人类总是无关紧要：人工智能不是用其他东西取代我们自己的智能，而是将智能——一种更不同的智能——带入我们的生活和工作。 在许多领域，尤其是在创意领域，人工智能将被人类用作增强自身能力的工具：比人工智能更多的增强智能。

艺术创作的很大一部分包括简单的模式识别和技术技能。而这正是许多人认为不那么有吸引力甚至可有可无的过程的一部分。这就是人工智能的用武之地。我们的感知方式、我们的语言和我们的艺术作品都有统计结构。学习这种结构是深度学习算法所擅长的。机器学习模型可以学习图像、音乐和故事的统计潜在空间，然后他们可以从这个空间中采样，创造出与模型在训练数据中看到的特征相似的新艺术作品。当然，这种采样本身并不是一种艺术创作行为。这仅仅是一个数学运算：该算法没有人类生活、人类情感或我们对世界的经验的基础；相反，它从与我们几乎没有共同之处的经验中学习。只有我们作为人类旁观者的解释才能赋予模型生成的意义。但在熟练的艺术家手中，算法生成可以被引导变得有意义和美丽。潜在空间采样可以成为赋予艺术家权力的画笔，增强我们的创造力，并扩展我们可以想象的空间。更重要的是，它可以通过消除对技术技能和实践的需求来使艺术创作更容易获得——建立一种新的纯粹表达媒介，将艺术与工艺分开。

Iannis Xenakis 是电子和算法音乐的有远见的先驱，在 1960 年代，在自动化技术应用于音乐创作的背景下，他精美地表达了同样的想法：

> 从繁琐的计算中解脱出来，作曲家能够致力于解决新音乐形式带来的一般问题，并在修改输入数据值的同时探索这种形式的角落和缝隙。 例如，他可能会测试从独奏家到室内管弦乐队，再到大型管弦乐队的所有乐器组合。 在电子计算机的帮助下，作曲家变成了一种飞行员：他按下按钮，输入坐标，并监督宇宙飞船在声音空间中航行的控制，穿越声波星座和星系，他以前只能通过 遥远的梦。

在本章中，我们将从各个角度探讨深度学习在增强艺术创作方面的潜力。 我们将回顾使用变分自动编码器和生成对抗网络的序列数据生成（可用于生成文本或音乐）、DeepDream 和图像生成。 我们会让您的计算机梦想出前所未有的内容； 也许我们也会让您梦想技术与艺术的交汇处的奇妙可能性。 让我们开始吧。

## 文本生成

在本节中，我们将探讨如何使用循环神经网络来生成序列数据。 我们将使用文本生成作为示例，但完全相同的技术可以推广到任何类型的序列数据：您可以将其应用于音符序列以生成新音乐，以及笔触数据的时间序列（例如， 艺术家在 iPad 上绘画时录制）以逐笔生成绘画，依此类推。

序列数据生成绝不仅限于艺术内容生成。 它已成功应用于语音合成和聊天机器人的对话生成。 谷歌于 2016 年发布的智能回复功能，能够自动生成一系列对电子邮件或短信的快速回复，由类似技术提供支持。

### 序列生成的生成式深度学习简史

2014 年底，即使在机器学习社区中，也很少有人见过 LSTM 的缩写。使用循环网络生成序列数据的成功应用直到 2016 年才开始出现在主流中。但是这些技术有着相当长的历史，从 1997 年 LSTM 算法的发展开始。这种新算法很早就被用来逐个字符地生成文本。

2002 年，当时在瑞士 Schmidhuber 实验室的 Douglas Eck 首次将 LSTM 应用于音乐生成，并取得了可喜的成果。Eck 现在是 Google Brain 的一名研究员，2016 年他在那里成立了一个名为 Magenta 的新研究小组，专注于应用现代深度学习技术来制作引人入胜的音乐。 有时，好的想法需要 15 年才能开始。

在 2000 年代末和 2010 年代初，Alex Graves 在使用循环网络生成序列数据方面做了重要的开创性工作。特别是，他在 2013 年应用循环混合密度网络使用笔位置的时间序列生成类似人类的笔迹的工作被一些人视为一个转折点。神经网络在那个特定时刻的这种特定应用让我想起了做梦的机器的概念，并且在我开始开发 Keras 时给了我很大的启发。格雷夫斯在 2013 年上传到预印服务器 arXiv 的 LaTeX 文件中留下了类似的注释掉的评论：“生成顺序数据是最接近梦想的计算机。”几年后，我们认为很多这些发展是理所当然的。但在当时，很难看到 Graves 的演示，也很难不因各种可能性而敬畏地走开。 2015 年至 2017 年间，循环神经网络成功用于文本和对话生成、音乐生成和语音合成。

然后在 2017-2018 年左右，Transformer 架构开始接管循环神经网络，不仅用于有监督的自然语言处理任务，还用于生成序列模型——特别是语言建模、词级文本生成。 生成式 Transformer 最著名的例子是 GPT-3，这是一个 1750 亿参数的文本生成模型，由初创公司 OpenAI 在一个惊人的大文本语料库上训练——包括大多数数字可用的书籍、维基百科和大部分爬行 整个互联网。 GPT-3 成为 2020 年的头条新闻，因为它能够为几乎任何主题生成听起来似乎合理的文本段落，这种能力已经引发了短暂的炒作浪潮，值得最火热的 AI 夏季。

### 如何生成序列数据？